## Try to convert the sound data into pcm samples

[https://www.atariarchives.org/dere/chapt07.php](https://www.atariarchives.org/dere/chapt07.php)

In [187]:
def get_channel_data(chn_data):
    channels = []
    for chn in raw_chns:
        channel = []
        for frame in chn.splitlines()[1:]:
            if not "BYTE" in frame:
                continue
            frame = frame.split()[1].split(',')
            channel += [[int(x,16) for x in frame]]
        channels += [channel]
    return channels

def get_channel_bytes(frame_data):
    channel = []
    for stval,frcnt,change,number in frame_data[:-1]:
        channel+=[stval]*frcnt
        for i in range(0,number-1):
            stval+=change
            stval=stval%255
            channel+=[stval]*frcnt
    return channel

def create_channels(chan_data):
    channel_bytes = []
    for frame_data in chan_data:
        channel_bytes+=[get_channel_bytes(frame_data)]
    return channel_bytes

def create_pcm_bytes(ch1,ch2,mult=150):
    pcm_bytes = []
    for i,x in enumerate(ch1):
        pcm_bytes+=[x]*mult
        if i >= len(ch2):
            pcm_bytes+=[ch2[-1]]*mult
        else:
            pcm_bytes+=[ch2[i]]*mult
    return pcm_bytes

In [186]:
0x0b*0x40

704

In [194]:
raw_chns = """
SL1F:
    .BYTE 18,4,0,0FF
	.BYTE 0,0
SL1A:
    .BYTE 0AF,4,0,0FF
	.BYTE 0,0
""".split(":")[1:]

# Make it
chan_data = get_channel_data(raw_chns)
channel_bytes = create_channels(chan_data)
print(len(channel_bytes[0]),len(channel_bytes[1]))
print(channel_bytes)
pcm_bytes = create_pcm_bytes(*channel_bytes,10)

# Play it
open("test.smp",'wb').write(bytes(pcm_bytes))
!play -t raw -r 15k -e unsigned-integer -b 8 -c 2 test.smp


1020 1020
[[24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 

In [136]:
!rm -f smp.raw
!arecord -v -f U8 -c 2 -t raw smp.raw


Recording raw data 'smp.raw' : Unsigned 8 bit, Rate 8000 Hz, Stereo
ALSA <-> PulseAudio PCM I/O Plugin
Its setup is:
  stream       : CAPTURE
  access       : RW_INTERLEAVED
  format       : U8
  subformat    : STD
  channels     : 2
  rate         : 8000
  exact rate   : 8000 (8000/1)
  msbits       : 8
  buffer_size  : 4000
  period_size  : 1000
  period_time  : 125000
  tstamp_mode  : NONE
  tstamp_type  : GETTIMEOFDAY
  period_step  : 1
  avail_min    : 1000
  period_event : 0
  start_threshold  : 1
  stop_threshold   : 4000
  silence_threshold: 0
  silence_size : 0
  boundary     : 9007199254740992000
^C
Aborted by signal Interrupt...


In [137]:
!play -t raw -r 8k -e unsigned-integer -b 8 -c 2 smp.raw


play WARN alsa: can't encode 0-bit Unknown or not applicable

smp.raw:

 File Size: 72.0k     Bit Rate: 128k
  Encoding: Unsigned PCM  
  Channels: 2 @ 8-bit    
Samplerate: 8000Hz       
Replaygain: off         
  Duration: 00:00:04.50  

In:100%  00:00:04.50 [00:00:00.00] Out:36.0k [      |      ]        Clip:0    
Done.


[52, 52]